In [1]:
%pip install datasets
%pip install evaluate

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install git+https://github.com/huggingface/accelerate

In [ ]:
import os
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import datasets

In [ ]:
from transformers import AutoTokenizer

In [ ]:
%pip install evaluate
import evaluate

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
# Import the dataset from hugging face. 
dataset = datasets.load_dataset("hupd-npe")

In [ ]:
dataset

In [ ]:
## TODO: SAMPLING METHOD
# Litigated to non-litigated ratio is 1:2.
# Year
# Year + CPC
# Get the train dataset
train_dataset = dataset['train']

# Convert the train dataset to a pandas DataFrame
accepted_df = train_dataset.to_pandas()

# Create a new column to indicate which rows will be included in our litigation dataset (1 for included, 0 for not included)
accepted_df["include_in_dataset"] = 0

# For each litigated patent, choose two other non-litigated patents to create a dataset
litigated_patents = accepted_df[accepted_df["npe_litigated_count"] > 0]

# Set the include_in_dataset column to 1 for the litigated patents
accepted_df.loc[litigated_patents.index, "include_in_dataset"] = 1

# Get the year of the litigated patents count
year_litigated = dict()
accepted_df['filing_date'] = pd.to_datetime(accepted_df['filing_date'])

# For each litigated patent, choose two other non-litigated patents to create a dataset
litigated_patents = accepted_df[accepted_df["npe_litigated_count"] > 0]

# Get the year of the litigated patents and then efficiently sample two non-litigated patents
for index, row in litigated_patents.iterrows():
    year = row["filing_date"].year
    if year in year_litigated:
        year_litigated[year] += 1
    else:
        year_litigated[year] = 1
# Initialize a list to store the indices of litigated patents to be excluded
excluded_litigated_indices = []

# Go over year_litigated and sample two non-litigated patents for each litigated patent
for year, count in year_litigated.items():
    # Sample from the same year as the litigated patent (if there are any) that have not been included in the dataset
    non_litigated_same_year = accepted_df[(accepted_df["filing_date"].dt.year == year) & (accepted_df["npe_litigated_count"] == 0) & (accepted_df["include_in_dataset"] == 0)]
    size = len(non_litigated_same_year)
    if size >= 2*count:
        non_litigated_same_year = non_litigated_same_year.sample(2*count, replace=False)
        accepted_df.loc[non_litigated_same_year.index, "include_in_dataset"] = 1
    else:
        # If there are not enough non-litigated patents, add the indices of the litigated patents from this year to the exclusion list
        litigated_same_year = accepted_df[(accepted_df["filing_date"].dt.year == year) & (accepted_df["npe_litigated_count"] > 0)]
        excluded_litigated_indices.extend(litigated_same_year.index.tolist())

# Set the include_in_dataset value to 0 for the excluded litigated patents
accepted_df.loc[excluded_litigated_indices, "include_in_dataset"] = 0

# Now, the dataset should only include litigated patents that have at least two non-litigated patents from the same year
included_patents = accepted_df[accepted_df["include_in_dataset"] == 1]

# Get the counts of litigated and non-litigated patents in the included patents
litigated_count = included_patents[included_patents["npe_litigated_count"] > 0]["npe_litigated_count"].count()
non_litigated_count = included_patents[included_patents["npe_litigated_count"] == 0]["npe_litigated_count"].count()

# Calculate the ratio
print(litigated_count, non_litigated_count)


In [ ]:

included_patents = accepted_df[accepted_df["include_in_dataset"] == 1]

# Get the counts of litigated and non-litigated patents in the included patents
litigated_count = included_patents[included_patents["npe_litigated_count"] > 0]["npe_litigated_count"].count()
non_litigated_count = included_patents[included_patents["npe_litigated_count"] == 0]["npe_litigated_count"].count()

# Calculate the ratio
print(litigated_count, non_litigated_count)

litigated_count = accepted_df[accepted_df["npe_litigated_count"] > 0]["npe_litigated_count"].count()
non_litigated_count = accepted_df[accepted_df["npe_litigated_count"] == 0]["npe_litigated_count"].count()
print(litigated_count, non_litigated_count)


In [ ]:
# Get the year and cpc_label of the litigated patents count
train_dataset = dataset['train']

# Convert the train dataset to a pandas DataFrame
accepted_df = train_dataset.to_pandas()

# Create a new column to indicate which rows will be included in our litigation dataset (1 for included, 0 for not included)
accepted_df["include_in_dataset"] = 0
accepted_df['filing_date'] = pd.to_datetime(accepted_df['filing_date'])
# For each litigated patent, choose two other non-litigated patents to create a dataset
litigated_patents = accepted_df[accepted_df["npe_litigated_count"] > 0]

# Set the include_in_dataset column to 1 for the litigated patents
accepted_df.loc[litigated_patents.index, "include_in_dataset"] = 1
year_cpc_litigated = dict()

# Get the year and cpc_label of the litigated patents and then efficiently sample two non-litigated patents
for index, row in litigated_patents.iterrows():
    year = row["filing_date"].year
    cpc_label = row["cpc_label"]
    if (year, cpc_label) in year_cpc_litigated:
        year_cpc_litigated[(year, cpc_label)] += 1
    else:
        year_cpc_litigated[(year, cpc_label)] = 1

# Initialize a list to store the indices of litigated patents to be excluded
excluded_litigated_indices = []

# Go over year_cpc_litigated and sample two non-litigated patents for each litigated patent
for (year, cpc_label), count in year_cpc_litigated.items():
    # Sample from the same year and cpc_label as the litigated patent (if there are any) that have not been included in the dataset
    non_litigated_same_year_cpc = accepted_df[(accepted_df["filing_date"].dt.year == year) & (accepted_df["cpc_label"] == cpc_label) & (accepted_df["npe_litigated_count"] == 0) & (accepted_df["include_in_dataset"] == 0)]
    size = len(non_litigated_same_year_cpc)
    if size >= 2*count:
        non_litigated_same_year_cpc = non_litigated_same_year_cpc.sample(2*count, replace=False)
        accepted_df.loc[non_litigated_same_year_cpc.index, "include_in_dataset"] = 1
    else:
        # If there are not enough non-litigated patents, add the indices of the litigated patents from this year and cpc_label to the exclusion list
        litigated_same_year_cpc = accepted_df[(accepted_df["filing_date"].dt.year == year) & (accepted_df["cpc_label"] == cpc_label) & (accepted_df["npe_litigated_count"] > 0)]
        excluded_litigated_indices.extend(litigated_same_year_cpc.index.tolist())

# Set the include_in_dataset value to 0 for the excluded litigated patents
accepted_df.loc[excluded_litigated_indices, "include_in_dataset"] = 0

# Now, the dataset should only include litigated patents that have at least two non-litigated patents from the same year and cpc_label
included_patents = accepted_df[accepted_df["include_in_dataset"] == 1]

# Get the counts of litigated and non-litigated patents in the included patents
litigated_count = included_patents[included_patents["npe_litigated_count"] > 0]["npe_litigated_count"].count()
non_litigated_count = included_patents[included_patents["npe_litigated_count"] == 0]["npe_litigated_count"].count()

# Calculate the ratio
print(litigated_count, non_litigated_count)

In [ ]:
dataset

In [ ]:
# Create train and test splits
# Train set should contain all the data up to year 2015 (use filing_date)
# Test set should contain all the data from year 2016 onwards (use filing_date)
train_set = dataset.filter(lambda example: int(example["filing_date"][0:4]) <= 2014)
test_set = dataset.filter(lambda example: int(example["filing_date"][0:4]) > 2014)

In [ ]:
#Baseline using HUPD pretrained distilroberta
tokenizer = AutoTokenizer.from_pretrained("HUPD/hupd-distilroberta-base")

# Roberta 
# tokenizer = AutoTokenizer.from_pretrained("roberta-base") 

In [ ]:
SECTION = "claims"
CPC_SECTION = "cpc_label"
EXAMINER_ID = "examiner_id"
# def tokenize_function(examples):
#     return tokenizer(examples[SECTION], truncation=True)

# def tokenize_function(examples):
#     # Concatenate CPC code with the text of the claims
#     text_with_cpc = [claim + " [CPC: " + cpc + "]" for claim, cpc in zip(examples[SECTION], examples[CPC_SECTION])]

def tokenize_function(examples):
    # Concatenate CPC code and examiner ID with the text of the claims
    text_with_cpc = [claim + " [CPC: " + cpc + "]" + " [examiner id: " + examiner_id + "]" for claim, cpc, examiner_id in zip(examples[SECTION], examples[CPC_SECTION], examples[EXAMINER_ID])]

    return tokenizer(text_with_cpc, truncation=True)

In [ ]:
train_set

In [ ]:
train_set_tokenized = train_set.map(tokenize_function, batched=True)
test_set_tokenized = test_set.map(tokenize_function, batched=True)

In [ ]:
# Set the label 0 if npe_litigated_count = 0 and 1 otherwise
train_set_tokenized = train_set_tokenized.map(lambda example: {"label": 0 if example["npe_litigated_count"] == 0 else 1})
test_set_tokenized = test_set_tokenized.map(lambda example: {"label": 0 if example["npe_litigated_count"] == 0 else 1})

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# Counter
from collections import Counter

Counter(test_set_tokenized["train"]["label"])

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np
from scipy.special import softmax

# accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    # Calculate probabilities for AUC
    probabilities = softmax(logits, axis=1)[:, 1]

    # Compute metrics
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='binary')
    auc = roc_auc_score(labels, probabilities)

    # Calculate confusion matrix and extract FN and FP
    tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()

    return {
        "accuracy": acc,
        "f1": f1,
        "auc": auc,
        "false_negatives": fn,
        "false_positives": fp,
    }

In [ ]:
id2label = {0: "NOT_LITIGATED", 1: "LITIGATED"}
label2id = {"NOT_LITIGATED": 0, "LITIGATED": 1}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "HUPD/hupd-distilroberta-base", num_labels=2, id2label=id2label, label2id=label2id
)

# model = AutoModelForSequenceClassification.from_pretrained(
#     "roberta-base", num_labels=2, id2label=id2label, label2id=label2id
# )

In [ ]:
train_set_tokenized

In [ ]:
training_args = TrainingArguments(
    output_dir="test",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",  # Change evaluation strategy to 'steps'
    eval_steps=500,  # Evaluate every 500 steps
    logging_steps=250,  # Log every 500 steps
    save_strategy="epoch",
    load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set_tokenized["train"],
    eval_dataset=test_set_tokenized["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
#Was not used, extra visualization and data 
first_layer_weights = model.roberta.embeddings.word_embeddings.weight.data
print(first_layer_weights)

In [ ]:
#Was not used, extra visualization and data 
import matplotlib.pyplot as plt

plt.hist(first_layer_weights.view(-1).cpu().numpy(), bins=100)
plt.title("Distribution of Weights in the First Layer")
plt.xlabel("Weight Value")
plt.ylabel("Frequency")
plt.show()


In [ ]:
#Was not used, extra visualization and data 
for name, param in model.named_parameters():
    print(name, param.size())
